In [ ]:
import pandas as pd
import numpy as np


# Load data
#pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["Germany", "Canada", "France", "Itary", "Japan", "United Kingdom", "United States America"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]

data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # 一人当たりGDP
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # 一人当たり資本
data['a'] = 1 - data['labsh']  # Capital share　－　資本シェア

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['c'] = (grouped_data['k_pc'].diff() * 100) * (data['a']) # 資本深化


grouped_data = data.groupby('countrycode')

data['t'] = data['g']  - data['c']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean',
                                       'a': "mean",'labsh': 'mean','c': 'mean','t': 'mean'})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['a']
summary['TFP Share'] = summary['labsh']
summary['Capital Deepening'] = summary['c']
summary['TFP growth'] = summary['t']

# Print output
#print(summary['Growth Rate'],summary['Capital Share'],summary['TFP Share'],summary['Capital Deepening'],summary['TFP growth'])
print(summary)

                    g         a     labsh         c         t  Growth Rate  \
countrycode                                                                  
AUS          1.724265  0.348140  0.651860  0.564797  1.159467     1.724265   
AUT          3.038354  0.365929  0.634071  1.409189  1.629165     3.038354   

             Capital Share  TFP Share  Capital Deepening  TFP growth  
countrycode                                                           
AUS               0.348140   0.651860           0.564797    1.159467  
AUT               0.365929   0.634071           1.409189    1.629165  
